In [1]:
#| default_exp audio

In [2]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')

In [3]:
#| export
import asyncio
from pydub import AudioSegment
import io
import subprocess

/usr/local/lib/python3.9/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
#| export

class AudioSlicer:
    def __init__(self, data=None, format="mp3"):
        self.format = format
        self.audio = AudioSegment.from_file(io.BytesIO(data), format=format) if data is not None else None

    
    @classmethod
    async def from_file(cls, file_path, format="mp3"):
        def read_file(file_path):
            with open(file_path, "rb") as file:
                return file.read()

        data = await asyncio.to_thread(read_file, file_path)
        return cls(data, format)

    async def export2file(self, export_path, start=None, end=None):
        def export(segment, export_path, format):
            segment.export(export_path, format=self.format)

        segment = self.slice(start, end)
        await asyncio.to_thread(export, segment, export_path)

    async def export_data(self, start=None, end=None):
        def export(segment, buffer):
            segment.export(buffer, format='mp3')
            return buffer.getvalue()

        segment = self.slice(start, end)
        buffer = io.BytesIO()
        return await asyncio.to_thread(export, segment, buffer)

    # slice remains synchronous as it's a simple in-memory operation
    def slice(self, start=None, end=None):

        if start is not None:
            start_millis = start * 1000
            end_millis = end * 1000
            audio = self.audio[start_millis:end_millis]
        else:
            print(start)
            audio = self.audio

        return audio


In [7]:
audio_obj = await AudioSlicer.from_file('/app/testdata/david_audio.webm','webm')

In [8]:
audio_obj.slice(0,10)